In [1]:
import matplotlib.pyplot as plt
from scipy.stats import binom
import numpy as np
import math
import scipy.stats as stats
# to use hedgehog, one needs to install two packages vose and hedgehog
# pip install git+https://github.com/MaxHalford/vose
# pip install git+https://github.com/MaxHalford/hedgehog
# you may also need to install graphviz to plot PGM
# conda install -c conda-forge python-graphviz 
import pandas as pd
from scipy.special import logsumexp
from IPython.display import Markdown as md
from random import randrange
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)
#  a hack to hide code from cell: https://github.com/damianavila/RISE/issues/32    

In [2]:
%%html
<style>
 .container.slides .celltoolbar, .container.slides .hide-in-slideshow {
    display: None ! important;
}
    
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
th, td {
  padding: 5px;
}
th {
  text-align: left;
}
</style>

In [3]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Foreword 

The following code are used for this lecture
  * most of the code have been adapted from https://github.com/aimacode/aima-python

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    
def g(n): return n.path_cost    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]



def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [5]:
FIFOQueue = deque

LIFOQueue = list

StackAgenda = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [6]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [7]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [8]:
# Some specific RouteProblems

romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})


r0 = RouteProblem('A', 'A', map=romania)
# r1 = RouteProblem('A', 'B', map=romania)
# r2 = RouteProblem('N', 'L', map=romania)
# r3 = RouteProblem('E', 'T', map=romania)
# r4 = RouteProblem('O', 'M', map=romania)

In [9]:
threeGrid = Map(
    {('A', 'B'):  1, ('A', 'D'): 1, ('B', 'C'): 1, ('B', 'E'): 1, ('C', 'F'): 1, 
     ('D', 'E'): 1, ('D', 'G'):  1, ('E', 'F'): 1, ('E', 'H'): 1, ('F', 'I'): 1, 
     ('G', 'H'): 1, ('H', 'I'):  1},
    {'A': ( -1, 1), 'B': (0, 1), 'C': (1, 1), 'D': (-1, 0), 'E': (0, 0), 
     'F': (1, 0), 'G': (-1, -1), 'H': (0, -1), 'I': (1, -1)})

# CS5010 Artificial Intelligence Principles
### Lecture 17 Uninformed Search 

##### Variants of uninformed searching algorithms
Lei Fang

University of St Andrews

# Last time

* Searching problem and algorithm
  * superimpose a tree on top of the state space graph
  
* Breadth first search
  * visit nodes based on hops from the root (initial state)

* Depth first search
  * dive into one branch and backtrack
  

# Today

* A general searching algorithm template
  * BFS and DFS as specific cases
  
* New variants of searching algorithms based on the template
  * Non-deterministic search
  * Uniform cost searching

# A general searching algorithm 


All algorithms we study follow a template
  * shouldn't be surprising: they all super-impose a search tree on the state space graph 
  * only differ in how the tree is constructed or super-imposed

Can be considered as the *mother* of all searching algorithms introduced here

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> and reorganise <font color="purple"><i>agenda</i></font><br>   
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;      # algorithms differ here!  <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>



* **agenda**: also called **open list**, **frontier**; agenda keeps all paths (potential solution) yet to be considered;
* **cyclic paths**: paths that contains cycles e.g. [A,B,A,B], [A,B,C,D,B]
* **exiting the search**: either a solution found (step 2) or exhausted all options and found nothing, i.e. empty agenda (step 6)
* **extended_list**: also called **closed list**; nodes that have already been extended (i.e. popped and extended step 1 and 3); this is optional; if not used, remove all lines with "[]" 

# DFS and BFS

Steps 0 to 4 are the same !
  * initialise with a one-element (initial state) agenda
  * keep popping from the agenda 
  * until a solution is found or nothing left in the agenda
  

![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/queuestack.png)

DFS and BFS only differ in step 5: **how the agenda is reorganised**
  * DFS: newly extended paths are added at the **front** of the **agenda**; agenda is a LIFO stack
  * BFS: newly extended paths are added at the **back** of the **agenda**; equivalently, agenda is a FIFO queue


DFS       |  BFS
:-------------------------:|:-------------------------:
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/dfsagenda.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfsagenda.png)

In [16]:
def my_breadth_first_search(problem, use_extendedList = True):
    agenda = FIFOQueue([Node(problem.initial)])
#   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
                        agenda.appendleft(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.appendleft(child)
    return failure

In [17]:
# Python implementation of DFS
def my_depth_first_search(problem, use_extendedList= False):
    agenda = ([Node(problem.initial)])
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       by default: pop() return the element at -1 (the end of the list)  
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       append at the end in Python has the same effect as adding at the front
#                       as pop() return from -1 by default
                        agenda.append(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.append(child)
    return failure

# Performance of DFS and BFS

DFS       |  BFS
:-------------------------:|:-------------------------:
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/dfscomplex.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfscomplex.png)


| | Complete| Optimal | Time complexity| Space complexity |
|:---|:---|:---|:---|:---|
| DFS| No | No | $O(b^d)$| $O(bd)$|
| BFS| Yes | No in general | $O(b^d)$| $O(b^d)$|

* $d$ is the depth (tiers) of optimal solution 
  * for DFS, you can swap with $m$ as it may not be lucky to start with the right branch
  * for BFS, $d$ is $s$
* $b$ is branching factor

*The figures are taken from https://inst.eecs.berkeley.edu/~cs188/su21/assets/notes/n1.pdf*

## Variants of searching algorithm

Various of new searching algorithms can be "**invented**" by modifying step 5: 

*how the agenda is reorganised*


# Non-deterministic searching: a mixture of both DFS and BFS

Instead of searching in some pre-determined fixed order
  * insert infront or back
  * one can carry out some random search

**Non-deterministic searching (NDS)**: randomly inject the newly extended paths to the existing agenda


|DFS       |  BFS |      NDS|
|:-------------------------:|:-------------------------:| :--------------------------:|
|![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/dfsagenda.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfsagenda.png) | ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/nondagenda.png)|

**Non-deterministic searching** pseudo-code

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> randomly insert <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> </b>
&nbsp;# <b>yay ! a new algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>


In [27]:
# Python implementation
def my_nondeterministic_search(problem, use_extendedList = True):
    agenda = list([Node(problem.initial)])
#   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       random insertion
                        agenda.insert(randrange(len(agenda)+1), child) 
        else:
            for child in expand(problem, node):  
                if not is_cycle(child):
#                   random insertion
                    agenda.insert(randrange(len(agenda)+1), child)
    return failure

# Non-deterministic search any good ?


**complete**: not guranteed! 
  * the worst scenario is it inserts all paths at the beginning (very unlikely but worst scenario); 
  * then reduce to DFS

**optimal**: not guranteed; 
  * again, the worst case is it may return a solution similar to DFS




**time complexity**: still $O(b^d)$; 
  * if you are unlucky, you still need to expand all nodes to find the solution; 
  * remember we care the worst case performance

**space complexity**: also $O(b^d)$; 
  * again if you are unlucky, you may insert everything to the back! 
  * reduce to BFS;


The above are **worst case** performance

But the **expected (or on average)** performance should be better ! 

Similar to the analysis of *quicksort* (another famous non-deterministic algorithm with 
  * a worst complexity of $O(N^2)$ 
  * but expected complexity $O(N\log N)$

# Demonstration on 3 by 3 grid problem

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/grid.png" width = "300" height="300"/></center>  

Formulate a route finding problem that starts from A and ends in B

In [28]:
rgrid0 = RouteProblem('A', 'B', map=threeGrid)
rgrid1 = RouteProblem('A', 'F', map=threeGrid) 

In [29]:
print("BFS's path for A to B is: ", path_states(my_breadth_first_search(rgrid0)))
print("BFS's path for A to F is: ", path_states(my_breadth_first_search(rgrid1)))

BFS's path for A to B is:  ['A', 'B']
BFS's path for A to F is:  ['A', 'B', 'C', 'F']


In [30]:
print("DFS's path for A to B is: ", path_states(my_depth_first_search(rgrid0)))
print("DFS's path for A to F is: ", path_states(my_depth_first_search(rgrid1, use_extendedList=False)))

DFS's path for A to B is:  ['A', 'D', 'G', 'H', 'E', 'B']
DFS's path for A to F is:  ['A', 'D', 'G', 'H', 'E', 'B', 'C', 'F']


### Nondeterministic search: each run might return a different solution

As expected, the results returned can be random
  * that's why not optimal (worst case scenario)
  * on average perform well though

In [35]:
# run 20 times of nondeterminisitc search
for i in range(20):
    print(path_states(my_nondeterministic_search(rgrid0)))

['A', 'B']
['A', 'B']
['A', 'D', 'E', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'D', 'E', 'B']
['A', 'D', 'E', 'B']
['A', 'D', 'G', 'H', 'E', 'F', 'C', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'B']
['A', 'D', 'E', 'B']
['A', 'B']


In [38]:
# run 20 times of nondeterminisitc search for A to F
for i in range(20):
    print(path_states(my_nondeterministic_search(rgrid1)))

['A', 'D', 'E', 'F']
['A', 'D', 'G', 'H', 'E', 'F']
['A', 'D', 'E', 'F']
['A', 'D', 'G', 'H', 'I', 'F']
['A', 'D', 'E', 'F']
['A', 'B', 'E', 'H', 'I', 'F']
['A', 'B', 'E', 'F']
['A', 'D', 'E', 'F']
['A', 'B', 'C', 'F']
['A', 'B', 'E', 'F']
['A', 'B', 'E', 'F']
['A', 'D', 'E', 'F']
['A', 'B', 'C', 'F']
['A', 'B', 'C', 'F']
['A', 'D', 'E', 'F']
['A', 'D', 'E', 'F']
['A', 'B', 'C', 'F']
['A', 'D', 'E', 'F']
['A', 'B', 'E', 'F']
['A', 'D', 'E', 'B', 'C', 'F']


#### Non-deterministic search on Romanian route 

* it finds the optimal solution (418) ! 
* although optimality is not guaranteed

In [51]:
# formulate the route finding problem: Arad to Bucharest
# use non-deterministic search to a path 20 times
r1 = RouteProblem('A', 'B', map=romania)
for i in range(20):
    node = my_nondeterministic_search(r1)
    print("Path:", path_states(node), "Cost:", g(node))

Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'R', 'P', 'B'] Cost: 418
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'R', 'P', 'B'] Cost: 418
Path: ['A', 'Z', 'O', 'S', 'F', 'B'] Cost: 607
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'Z', 'O', 'S', 'F', 'B'] Cost: 607
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'R', 'P', 'B'] Cost: 418
Path: ['A', 'S', 'R', 'P', 'B'] Cost: 418
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'T', 'L', 'M', 'D', 'C', 'P', 'B'] Cost: 733
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'Z', 'O', 'S', 'F', 'B'] Cost: 607
Path: ['A', 'S', 'F', 'B'] Cost: 450
Path: ['A', 'S', 'F', 'B'] Cost: 450


# Another variant : Uniform Cost Search (UCS)


So far we have seen the following **agenda** re-organisation methods:
  * DFS: insert in front
  * BFS: insert in back
  * non-deterministic: random insertion



Another variant **Uniform Cost Searching** (UCS): **sort the agenda based on the path costs !**
  * why sorting ? not every path in the agenda should be treated equally
  * focus on the promising ones first!

UCS pseudo-code:


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> then sort the agenda based on the path costs</b><br>
&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;       # <b>yay ! another algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>

# Implementation of UCS

Sorting the agenda in each iteration is not ideal 
  * best sorting complexity is $O(N\log N)$, $N$ is the size of the agenda
  * becomes expensive for each iteration! 

Note the agenda has been sorted in previous iteration
  * insert new extended paths to their correct place rather than sorting the whole agenda again
  * e.g. like one step of *insertion sort*: the complexity of insertion is: $O(N)$ though
  
Priority Queue (PQ) is suitable for this kind of task: the complexity for insertion is $O(\log N)$; 
  * The underlying data structure of a PQ is usually
    * Heap tree
    * or Balanced binary search tree, say AVL tree
 

In [52]:
# Python implementation of uniform cost search
def my_uniform_cost_search(problem, use_extendedList= True):
    agenda = PriorityQueue([Node(problem.initial)], key=g)
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       PQ.pop() returns the node with the least cost 
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       PriorityQueue taks care of this by inserting to 
#                       an appropriate location that is efficient for further operation
                        agenda.add(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.add(child)
    return failure

# UCS walk through

Romania routing problem: find a route between Arad to Bucharest
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png" width = "900" height="300"/></center>  

<font color='blue'><b>Step 0</b></font>: **initialisation:**

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS0.png" width = "50" height="300"/></center>  


--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list): initially $[A]$ is in the agenda (still yet to be checked and extended)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 1</b></font>:

Extend the first node in the agenda (which is **A**);

The next one to be popped and extended is **Z** (75), the most promising path 


--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS1.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 2</b></font>:

After extended **Z**

The next one to be popped, checked and extended becomes **T**

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS2.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 3</b></font>:

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS3.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 4</b></font>:

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS4.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 5</b></font>:

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS5.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 6</b></font>:

--------------------------
<font color='blue'><b>Agenda status</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS6.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 7</b></font>:

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS8.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Step 8</b></font>:


Note that one solution has been found and added to the agenda: [A,S,F,B] (450) 
  * the solution returned by BFS !


**BUT** it is not returned yet (we don't do early check as per algorithm)
  * why ? there might be other better options out there !

The next to be extended is [A,S,O] (291) actually!

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS9.png" width = "800" height="300"/></center>

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Fast forward a few steps</b></font>:

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS9_.png" width = "800" height="300"/></center>  


--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Next step</b></font>:

Note the final return solution has been found and inserted to agenda ! **[A,S,R,P,B]** with optimal cost **418**

**BUT** we cannot return it yet !
  * next to be popped and checked is **[A,S,O,Z] (362)**! (the most promising one in the agenda)

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS10.png" width = "800" height="300"/></center> 


--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Final return step</b></font>:

After we open all other promising routes, we are finally about to check **[A,S,R,P,B]** (418)
  * all other routes are officially worse
--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS11.png" width = "800" height="300"/></center>  


--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

# Demonstration of UCS

3 by 3 grid problem: UCS returns the optimal solution

In [53]:
print(path_states(my_uniform_cost_search(rgrid0, use_extendedList=False)))
print(path_states(my_uniform_cost_search(rgrid1, use_extendedList=False)))

['A', 'B']
['A', 'B', 'C', 'F']


Romania routing problem: find a route between Arad to Bucharest
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png" width = "900" height="300"/></center>  

In [54]:
print("The path found by UCS:", path_states(my_uniform_cost_search(r1, use_extendedList=False)))
print(g(my_uniform_cost_search(r1, use_extendedList=False)))

The path found by UCS: ['A', 'S', 'R', 'P', 'B']
418


In [58]:
print("The cost of the path found by UCS is:" , g(my_uniform_cost_search(r1, use_extendedList=False)))
print("The cost of the path found by BFS is:", g(my_breadth_first_search(r1, use_extendedList=False)))
print("The cost of the path found by DFS is:", g(my_depth_first_search(r1, use_extendedList=False)))
print("The cost of the path found by NDS is:", g(my_nondeterministic_search(r1, use_extendedList=False)))

The cost of the path found by UCS is: 418
The cost of the path found by BFS is: 450
The cost of the path found by DFS is: 733
The cost of the path found by NDS is: 450


Again, UCS has found the optimal solution between Arad to Bucharest (418 cost)
  * remember BFS's solution is not optimal (but with the least transitions)
  
Can we conclude UCS is **optimal** ?  
  * inventing an algorithm is *easy* but proving it works as desired is a bit harder

# UCS is optimal (?) !


Does the path returned by UCS is always **optimal** ?

Let's consider the case that extended_list is **not** in use first:
  * only cyclic solutions are discarded
  * it means extended nodes will be extended again and put back to the agenda 
  * the agenda is *complete* in the sense all possible solutions are kept (no early rejection) comparing with the case that extension_list in use
  

UCS will return the **first** path that **passes the goal test**
  * when the popped path ends with a goal state, it is returned immediately
  

To prove **optimality**, one only needs to show all other paths left in the agenda **cannot** beat the **first** popped goal path 



Take a look at the agenda the moment it is about to return a solution

$$\text{agenda} = [(P_1,c_1), (P_2, c_2), (P_3, c_3), \ldots ]$$
  * $P_i$ path, $c_i$ is $P_i$'s cost

* as the agenda is sorted: $$c_1 \leq c_2 \leq c_3 \leq \ldots$$

$P_1$ ends with a goal state, UCS is about to return $P_1$ as a solution

* all other paths $P_2, P_3, \ldots$ are worse than $P_1$: $c_1 \leq c_2 \leq c_3 \leq \ldots$!

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS11.png" width = "800" height="300"/></center>  


* if there is some $P_i$ ends with a goal state (B here in the figure above): they **cannot beat** $P_1$ 
  * *so do we need to keep redundant paths that end with the same state ?* probably not!
* and for other paths with non goal state, they are more hopeless
  * action costs are non-negative ! their costs will only get worse (if we continue the while loop)

Conclusion, we are safe to stop here! **UCS is optimal** *when extended_list is not in use*

### How about using extended_list ?

Does use extended_list affact the optimality ?

What extended_list do? 
  * it will stop extending already extended nodes

Take a look at the agenda at any iteration (not immediately before return)

$$\text{agenda} = [(P_1,c_1), (P_2, c_2), (P_3, c_3), \ldots ]$$
* the agenda is sorted, so we know $$c_1 \leq c_2 \leq c_3 \leq \ldots$$

$P_1= [A,B,C,D]$ is popped next and $D$ is extended, then $D$ is added to extended list

The question is: whether further *paths ending with $D$* can be **exempted** from extension (and not affect optimality)?
  * for example, when we later extend $P_2 = [\ldots, D]$, and $P_2$ happen to end with $D$
  * can we skip $P_2$'s extension ?

* yes! all later paths (no matter ending with $D$ or not) will have costs $>c_1$
  * only lead to inferior solution!
* no point extending them


# UCS is optimal ~~~(?)~~~ !

Key observation: the optimal path from $A$ to $G$ via $D$ is 
  * the optimal path from $A$ to $D$ (the first one found by UCS!)
  * plus the optimal path from $D$ to $G$
  * we only need to remember the best path to each node ! which is guaranteed by UCS
    * this is called **dynamic programming principle**  

  
Conclusion: **UCS is optimal** regardlessly whether extended list is used or not

# Modified UCS based on dynamic programming principle

Based on the observation, we can add another small agenda reorganisation step to further improve efficiency
  * we only need to keep one path (with the lowest cost) in agenda (remove those ones with inferior costs)
  * less paths to consider in agenda, slightly better efficiency 

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;then sort <font color="purple"><i>agenda</i></font> based on the costs <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;and for paths ending with the same node only keep the optimal one</b>
&nbsp; &nbsp; &nbsp;        <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>

# Question 

What is the relationship between UCS and BFS ?

Can you implement a BFS algorithm by using UCS or with PriorityQueue as agenda?

# Performance of UCS

BFS       |  UCS
:-------------------------:|:-------------------------:
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/bfscomplex.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/ucscomplex.png)

**Complete**: UCS is similar to BFS, it pushes into the tree (based on path cost) 
  * so yes! if there is a solution, it will return it

**Optimal**: yes! it will return the best solution; we have shown it in previous slides at length!



<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/ucscomplex.png" width = "500" height="300"/></center>  


**Time complexity**: UCS is similar to BFS, it pushes into the tree (based on path cost)
  * the complexity is determined by the number of nodes expanded or goal checked (critical step)
  * denote $C^{*}$ the optimal cost, $\epsilon$ the lower bound of action costs
  * we can roughly estimate the optimal depth $d = C^{*}/\epsilon$ (or tiers in the diagram)
  * complexity is $O(b^{C^{*}/\epsilon})$ (compare with $O(b^d)$ for BFS: where $C^{*}=d$ and $\epsilon=1$)


**Space complexity**: again similar to BFS, it needs to keep all those tree nodes that it has "pushed into"
  * $O(b^{C^{*}/\epsilon})$
  
*The diagram is taken from https://inst.eecs.berkeley.edu/~cs188/su21/assets/notes/n1.pdf*

# A quick summary of uninformed seach algorithms discussed so far

  | | Complete| Optimal | Time complexity| Space complexity | agenda reorganisation
|:---|:---|:---|:---|:---|:---|
| DFS| No | No | $O(b^d)$| $O(bd)$| paths added to the front agenda; or stack agenda|
| BFS| Yes | No in general | $O(b^d)$| $O(b^d)$| paths added to the back of agenda; or queue agenda |
| NDS| No | No  | $O(b^d)$| $O(b^d)$| paths randomly inserted to the agenda |
| UCS| Yes | Yes | $O(b^{C^{*}/\epsilon})$| $O(b^{C^{*}/\epsilon})$| paths added agenda then sort the agenda based on path costs (optional: trim down agenda by only retaining the most promising ones, i.e. if there are multiple paths ending with the same node, keep the one with the optimal cost); or PriorityQueue agenda|

We have an optimal searching algorithm now, but not efficient (still exponential complexity)! 

Can we do better ?
  * next week's topic! *informed search*

# Summary

* A general searching algorithm template
* Searching algorithm variants
  * Non-deterministic search
  * Uniform cost searching